In [1]:
import numpy as np

In [71]:
# check if current pairing is equilibrium
def is_steady_state(buyer_product_values, seller_product_values, pairings, epsilon=0):
    for buyer_index in range(buyer_product_values.size):
        buyer_point = buyer_current_partner = seller_point = seller_current_partner = np.nan

        # if the buyer has not made a deal, its max acceptible price is its product_value
        # if it has made a deal, its max acceptible price is its current price
        try:
            buyer_current_partner = np.nanargmax(pairings[buyer_index,:])
            buyer_point = pairings[buyer_index, buyer_current_partner]
        except ValueError:
            buyer_point = buyer_product_values[buyer_index]

        for seller_index in range(seller_product_values.size):            
            # if the seller has not made a deal, its min acceptible price is its product_value
            # if it has made a deal, its min acceptible price is its current price
            try:
                seller_current_partner = np.nanargmax(pairings[:, seller_index])
                seller_point = pairings[seller_current_partner, seller_index]
            except ValueError:
                seller_point = seller_product_values[seller_index]

            # if there is a deal to be made, return false
            if buyer_point < seller_point + epsilon: continue
            return False
        
    return True

In [136]:
trials = 1000
precision = 0
epsilon = 10**-precision

# the max prices buyers are willing to pay, sorted from strong to weak
buyer_product_values = np.sort(np.random.choice(
    np.arange(0,5,epsilon),
    size=np.random.randint(1,5)
))[::-1]
 # the min prices sellers are willing to accept, sorted from strong to weak
seller_product_values = np.sort(np.random.choice(
    np.arange(0,5,epsilon),
    size=np.random.randint(1,5)
))[::-1]

results_matrix = np.empty(shape=(buyer_product_values.size, seller_product_values.size, trials))
results_matrix[:] = np.nan

for trial in range(trials):
    # matrix of successful deals: buyer i sells to seller j at price k
    pairings = np.empty((buyer_product_values.size, seller_product_values.size,))
    pairings[:] = np.nan
    # randomly generate pairings and random prices
    buyer_permutation = np.random.permutation(np.arange(buyer_product_values.size))
    seller_permutation = np.random.permutation(np.arange(seller_product_values.size))
    for i in range(np.min([buyer_product_values.size, seller_product_values.size])):
        if buyer_product_values[buyer_permutation[i]] >= seller_product_values[seller_permutation[i]] + epsilon:
            pairings[buyer_permutation[i], seller_permutation[i]] = np.random.choice(
                np.arange(seller_product_values[buyer_permutation[i]], buyer_product_values[buyer_permutation[i]] + epsilon, epsilon)
            )
    
    if is_steady_state(buyer_product_values, seller_product_values, pairings, epsilon):
        np.copyto(results_matrix[:,:,trial], pairings)
        # print(pairings)

print("min, max")
print(np.nanmin(results_matrix, axis=2))
print(np.nanmax(results_matrix, axis=2))
print("count")
print(trials * (1 - np.nanmean(np.isnan(results_matrix), axis=2)))

min, max
[[nan  3.]
 [nan nan]
 [nan nan]]
[[nan  3.]
 [nan nan]
 [nan nan]]
count
[[  0. 329.]
 [  0.   0.]
 [  0.   0.]]


C:\Users\brand\AppData\Local\Temp\ipykernel_17248\1150718183.py:37: RuntimeWarning: All-NaN slice encountered
  print(np.nanmin(results_matrix, axis=2))
C:\Users\brand\AppData\Local\Temp\ipykernel_17248\1150718183.py:38: RuntimeWarning: All-NaN slice encountered
  print(np.nanmax(results_matrix, axis=2))


In [75]:
def find_clearing_price(buyer_product_values, seller_product_values, search_epsilon):
    clearing_max = np.max(buyer_product_values)
    clearing_min = 0
    while True:
        # print(clearing_min, clearing_max)
        clearing_price = (clearing_max + clearing_min)/2
        q_demanded = np.sum(buyer_product_values > clearing_price + search_epsilon)
        q_supplied = np.sum(seller_product_values < clearing_price - search_epsilon)
        if clearing_max - clearing_min <= search_epsilon or q_demanded == q_supplied:
            break
        if q_demanded > q_supplied:
            clearing_min = clearing_price + search_epsilon
        elif q_demanded < q_supplied:
            clearing_max = clearing_price - search_epsilon
    if q_demanded == 0 or q_supplied == 0: return None
    return clearing_price

In [76]:
# Check if clearing prices constitute steady state

trials = 1000
precision = 2
epsilon = 10**-precision

for trial in range(trials):
    # the max prices buyers are willing to pay, sorted from strong to weak
    buyer_product_values = np.sort(np.random.choice(
        np.arange(0,5,epsilon),
        size=np.random.randint(1,5)
    ))[::-1]
    # the min prices sellers are willing to accept, sorted from strong to weak
    seller_product_values = np.sort(np.random.choice(
        np.arange(0,5,epsilon),
        size=np.random.randint(1,5)
    ))[::-1]

    # find clearing price where quantity supplied = quantity demanded via binary search
    clearing_price = find_clearing_price(buyer_product_values, seller_product_values, search_epsilon=epsilon)
    if clearing_price == None:
        # print(f"No clearing price found for trial {trial} where:")
        # print("buyer_product_values:")
        # print(buyer_product_values)
        # print("seller_product_values:")
        # print(seller_product_values)
        # print()
        continue
    clearing_price = np.round(clearing_price, decimals=precision)

    # matrix of successful deals: buyer i sells to seller j at price k
    pairings = np.empty((buyer_product_values.size, seller_product_values.size,))
    pairings[:] = np.nan
    # randomly pair eligible buyers and sellers and clearing_price
    buyer_indices = np.argwhere(buyer_product_values > clearing_price)
    seller_indices = np.argwhere(seller_product_values < clearing_price)
    for i in range(np.min([buyer_indices.size, seller_indices.size])):
        pairings[buyer_indices[i], seller_indices[i]] = clearing_price
    
    # if not a steady state, print that it didn't work
    if not is_steady_state(buyer_product_values, seller_product_values, pairings, epsilon=epsilon):
        print(f"The clearing price of {clearing_price} did not produce a steady state in trial {trial} where:")
        print("buyer_product_values:")
        print(buyer_product_values)
        print("seller_product_values:")
        print(seller_product_values)
        print("buyer_indices", buyer_indices)
        print("seller_indices", seller_indices)
        print("pairings:")
        print(pairings)
        print()

The clearing price of 3.58 did not produce a steady state in trial 28 where:
buyer_product_values:
[4.77 3.59 3.58 3.58]
seller_product_values:
[1.58]
buyer_indices [[0]
 [1]]
seller_indices [[0]]
pairings:
[[3.58]
 [ nan]
 [ nan]
 [ nan]]

The clearing price of 4.29 did not produce a steady state in trial 164 where:
buyer_product_values:
[4.31 4.31 2.97]
seller_product_values:
[0.61]
buyer_indices [[0]
 [1]]
seller_indices [[0]]
pairings:
[[4.29]
 [ nan]
 [ nan]]

The clearing price of 3.64 did not produce a steady state in trial 223 where:
buyer_product_values:
[4.85 4.32 3.65 2.28]
seller_product_values:
[4.83 2.51 1.64]
buyer_indices [[0]
 [1]
 [2]]
seller_indices [[1]
 [2]]
pairings:
[[ nan 3.64  nan]
 [ nan  nan 3.64]
 [ nan  nan  nan]
 [ nan  nan  nan]]

The clearing price of 0.78 did not produce a steady state in trial 290 where:
buyer_product_values:
[4.19 0.69 0.23]
seller_product_values:
[4.67 0.84 0.77 0.61]
buyer_indices [[0]]
seller_indices [[2]
 [3]]
pairings:
[[ nan  na